A lot of effort in solving any machine learning problem goes into preparing the data. Though the data that's shared with you is curated, you still need to write code to read and iterate through the dataset. The first section shows you how to create a pytorch dataset. It essentially implements two methods, __getitem__ and __len__. 

please make sure the following packages are installed:


1. cv2: For reading images and transforms
2. pandas: For csv parsing
3. pytorch: A framework that makes training neural networks straight forward

### Mounting the data on collab
Add the zip file to a folder on gdrive. You'll be asked to authenticate to mount the folder.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
# This is path that has training data on my google drive.
!ls /content/gdrive/MyDrive/Plaksha_Assignment_Qure/

ls: cannot access '/content/gdrive/MyDrive/Plaksha_Assignment_Qure/': No such file or directory


In [41]:
path="/content/gdrive/MyDrive/consolidation_train_gt (1).csv"

In [42]:
df=pd.read_csv(path)

In [43]:
df.head()

,filename,consolidation,consolidation-left,consolidation-right
0,19047,0,0,0
1,17924,0,0,0
2,11658,0,0,0
3,10733,0,0,0
4,15041,0,0,0


Replace the path with your own path and run this command the first time to extract the images. Note that this will unzip at the root path, if you want to extract at a different location, use -d

In [46]:
!unzip  /content/gdrive/MyDrive/cxr_plaksha_assignment_qure_1.zip

Archive:  /content/gdrive/MyDrive/cxr_plaksha_assignment_qure_1.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/gdrive/MyDrive/cxr_plaksha_assignment_qure_1.zip or
        /content/gdrive/MyDrive/cxr_plaksha_assignment_qure_1.zip.zip, and cannot find /content/gdrive/MyDrive/cxr_plaksha_assignment_qure_1.zip.ZIP, period.


In [ ]:
# Verify that the zip file is extracted correctly.
!ls cxr_plaksha_assignment_qure | wc -l

20000


## Implementing a torch dataset and a dataloader(iterator on the dataset).

In [ ]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

In [ ]:
# if you don't have pytorch installed in your collab environment, install it using !pip install torch
print(torch.__version__)

1.10.0+cu111


In [ ]:
# A helper function to read png images using cv2.
def read_image_from_path(path):
    try:
        return cv2.imread(path, 0)
    except Exception as e:
        print(f"error in fpath {path}")

A basic implementation of a torch dataset. 

In [ ]:
class cls_dataset(Dataset):
    """A torch classification dataset class which returns each item in the form of dict consisting of keys idx, input, target.
    Transforms are applied on input images.
    """

    def __init__(self, images_path, ground_truth_path):
        self.images_path = images_path
        self.gt_df = pd.read_csv(ground_truth_path, index_col="filename", usecols=["filename", "consolidation"])
        self.transforms = ToTensor()

    def __len__(self):
        return len(self.gt_df.index)

    def __getitem__(self, index):

        idx = self.gt_df.index[index]

        input = self._get_input(idx)
        input = self.transforms(input)

        targets = self._get_target(idx)

        return {"idx": idx, "input": input, "target": targets}

    def _get_input(self, idx):
        filepath = os.path.join(self.images_path, str(idx) + ".png")
        return read_image_from_path(filepath)

    def _get_target(self, idx):
        return self.gt_df.loc[idx, "consolidation"]

### Once the dataset is implemented, you can use the torch.dataloader api to create smart iterators that you can use to automatically batch to process images parallely. In the example below, we are using a dataset of 4. 

### **Data Loader**

In [ ]:
images_path = "cxr_plaksha_assignment_qure"
ground_truth_path = "/content/gdrive/MyDrive/Plaksha_Assignment_Qure/consolidation_train_gt.csv"
train_ds = cls_dataset(images_path, ground_truth_path)
for batch in data.DataLoader(train_ds, batch_size=4):
    # print(batch)
    for i in range(0,4):
        print(f"for index {batch['idx'][i]} target is {batch['target'][i]}")
        image=torch.squeeze(batch['input'][i])
        plt.figure(i)
        plt.imshow(image, cmap='gray')
    break

FileNotFoundError: ignored

In [ ]:
#Import various libraries for CNN based image classification

from torchsummary import summary
import PIL
import sys
import torch
from time import time
import torchvision
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.autograd import Variable
import torchvision.transforms as transforms

# I have used Efficientnet3 pre-trained model for the classification task
from efficientnet_pytorch import EfficientNet


In [ ]:
# Set the learning rate

learning_rate=1e-4

In [ ]:
import torchvision.transforms as transforms

In [ ]:
# Define a transform operation that applies transformations to an image

transform_train = transforms.Compose([transforms.Resize((60,60)),transforms.RandomApply([
      transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip()],0.7),
		transforms.ToTensor()])

In [ ]:
# Create batches of size 32 each from the training dataset

training_generator = data.DataLoader(train_ds,shuffle=True,batch_size=32,pin_memory=True)

In [ ]:
# Enable GPU computation

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

### **Importing the model**

In [ ]:
# Instantiate Efficientnet3 

model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)

In [ ]:
# Load the model to device

model.to(device)

In [ ]:
# Instantiate Efficientnet3 

model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)

In [ ]:
# Make crossentropyloss as the criterion, set a learning rate decay and use Adam or weight update

criterion = nn.CrossEntropyLoss()
lr_decay=0.99
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Create a folder in the stat946winter2021 directory to save Weights

PATH_SAVE='./Weights/'
if(not os.path.exists(PATH_SAVE)):
    os.mkdir(PATH_SAVE)

In [ ]:
#Create a class list

eye = torch.eye(2).to(device)
classes=[0,1]

In [ ]:
# Create lists to record accuracy and loss and set the number of epochs ( I got 11 as the optimal number of epochs)

history_accuracy=[]
history_loss=[]
epochs = 11

In [ ]:
# Train the model

for epoch in range(epochs):  
    running_loss = 0.0
    correct=0
    total=0
    class_correct = list(0. for _ in classes)
    class_total = list(0. for _ in classes)
    
    for i, data in enumerate(training_generator, 0):
        inputs, labels = data
        t0 = time()
        inputs, labels = inputs.to(device), df.consolidation.to(device)
        labels = eye[labels]
        optimizer.zero_grad()
        #torch.cuda.empty_cache()
        outputs = model(inputs)
        loss = criterion(outputs, torch.max(labels, 1)[1])
        _, predicted = torch.max(outputs, 1)
        _, labels = torch.max(labels, 1)
        c = (predicted == labels.data).squeeze()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        accuracy = float(correct) / float(total)
        
        history_accuracy.append(accuracy)
        history_loss.append(loss)
        
        loss.backward()
        optimizer.step()
        
        for j in range(labels.size(0)):
            label = labels[j]
            class_correct[label] += c[j].item()
            class_total[label] += 1
        
        running_loss += loss.item()
        
        print( "Epoch : ",epoch+1," Batch : ", i+1," Loss :  ",running_loss/(i+1)," Accuracy : ",accuracy,"Time ",round(time()-t0, 2),"s" )
    for k in range(len(classes)):
        if(class_total[k]!=0):
            print('Accuracy of %5s : %2d %%' % (classes[k], 100 * class_correct[k] / class_total[k]))
        
    print('[%d epoch] Accuracy of the network on the Training images: %d %%' % (epoch+1, 100 * correct / total))
    
    if epoch%10==0 or epoch==0:
        torch.save(model.state_dict(), os.path.join(PATH_SAVE,str(epoch+1)+'_'+str(accuracy)+'.pth'))
        
torch.save(model.state_dict(), os.path.join(PATH_SAVE,'Last_epoch'+str(accuracy)+'.pth'))


### **Confidence scores**

In [ ]:
def predict_image(image):
    image_tensor = test_transforms(image)
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index